In [21]:
import ee
import ee.batch
import pyperclip
import time

In [22]:
# Initialize the Earth Engine library.
ee.Authenticate()
ee.Initialize(project='pixelcampione-satelital2map')


In [23]:
import ee.data


satellital_ids = []
elevation_ids = []

In [24]:
import ee.batch


def export_elevation(dataset_image,area_of_interest, filename, copy: bool = True):
    dataset = ee.image.Image(dataset_image)

    selected = dataset.select('elevation')
    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description': 'elevation_to_drive',
        'fileNamePrefix': filename,
        'scale': 90,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    elevation_ids.append(task.id)


def export_rgb(image: ee.image.Image,area_of_interest, filename, copy: bool = True):
        
    # Select the RGB bands
    selected = image.select(['B4', 'B3', 'B2'])

    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description':'rgb_to_drive',
        'fileNamePrefix': filename,
        'scale': 30,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    satellital_ids.append(task.id)

In [25]:
center = {
    'longitude': -120.0,
    'latitude': 40.0
}
offset = 0.10

# 5 x 5
lat_mult = 1
lon_mult = 3
index = {
    'start_lat': 6*lat_mult,
    'start_lon': 6*lon_mult,
    'end_lat': 6*(lat_mult+1),
    'end_lon': 6*(lon_mult+1)
}
# Me quede en 145
temp = 289

In [26]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset - 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset - 2*offset*i])
        # Call the function with your area of interest and desired filename prefix.
        export_elevation("CGIAR/SRTM90_V4",start, f'elevation_{counter}')
        counter += 1

        print("j = ",j)
    print("i = ",i)

print("-------------------------ELEVATION FINISHED-------------------------")

Task ID: J6ZC5JRMZWVUDID2AU4JTJBI
j =  12
Task ID: HTD3LCXVB2JAIRXEB7BT5O6V
j =  13
Task ID: A4UXKN3RD7BUQXHAWQRMQDKP
j =  14
Task ID: S6IAPQJWNKKPWZYHMCFLTLGV
j =  15
Task ID: KAJQUCPVSLB5X7FWYVMIUSIB
j =  16
Task ID: CMP6QEIAEVF63PQGBOEKRCXW
j =  17
i =  6
Task ID: XPDQYTVMETXOUC4VRBQZDCJ4
j =  12
Task ID: XAWKUYQT5P2DIN4BQQUPIFV5
j =  13
Task ID: PM7SVXBPBDTG3CVDKRC4WVG4
j =  14
Task ID: Z7LGBODE4BSV6HDX77J2N3YK
j =  15
Task ID: VTQH2HXUNBSXSTZHRFSJVXJH
j =  16
Task ID: MRYKVCTKQT5RXVLTCDQ3AXWE
j =  17
i =  7
Task ID: SHBDAURHOD5JHNDGS4G6TNLN
j =  12
Task ID: B6AS7HEK4O77U3RUYIAHX6Z5
j =  13
Task ID: DVVAXRAQRYPBDKP32AOB3PHD
j =  14
Task ID: PFB3JLBQFLNLS5IMXLLYVZYU
j =  15
Task ID: PGRVY2CVXVWPKEMS5Z2HCGCP
j =  16
Task ID: WMYYMJBLGLTXZFYSDR72NHFY
j =  17
i =  8
Task ID: T3EZFPQSSYI36GOKOW5TXJEB
j =  12
Task ID: VWSSPXGJFTESOLJQWMZNC7NO
j =  13
Task ID: P6VJHFGKKHBQCPBV7ICOTRVW
j =  14
Task ID: TUFOZM46NQJW3JFKQOSS4DAC
j =  15
Task ID: AHUZZUMIXUJ25OPKZPUTGJMR
j =  16
Task ID: AD2N

In [28]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset + 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset + 2*offset*i])

        # Convert ImageCollection to Image
        image = ee.imagecollection.ImageCollection("LANDSAT/LC09/C02/T1").filterBounds(start).mosaic()
        
        # Call the function with your area of interest and desired filename prefix.
        export_rgb(image, start, f'satellite_{counter}')
        counter += 1
        

        print("j = ",j)
    print("i = ",i)

print("-------------------------SATELLITE FINISHED-------------------------")

Task ID: R2QAL7WIFFLMCB6EPBI4A6TJ
j =  12
Task ID: OWHP6WDGFWUD44XBFDQFLMXD
j =  13
Task ID: LXR7KPOOMZFDU5OCN27YZETT
j =  14
Task ID: NMJ2LQN3NZWQMHBEWMSXBAEX
j =  15
Task ID: B4PKSBDQC5XHWBXXBNTID25U
j =  16
Task ID: ZXRJHQNDAVNUBAWQZ7XO5QW4
j =  17
i =  6
Task ID: AS3Q7PQU4MMCPDJ4GOSUESCD
j =  12
Task ID: OLLRQRTLJB2HXZINWDF2UYGL
j =  13
Task ID: 3L3VQMPOWNLTTZBZDJ43ZS4H
j =  14
Task ID: 65B3C3JKP2HP6CY2OTAYMI4K
j =  15
Task ID: ETOJBO4QICHFJGBB6ICJSOXX
j =  16
Task ID: LIHDLDPFSL5DPTFQKN4IBVKT
j =  17
i =  7
Task ID: 5OL73QYPD64QJCS73LRBYV57
j =  12
Task ID: KLOBBFS6E2JGE2DAD6MT47IP
j =  13
Task ID: PP32FSDXLQVSMONUEBUAXVZR
j =  14
Task ID: LDDA24Q3VSBR324CORURYZTU
j =  15
Task ID: 7JNNZ3QO3JH53P4DGYPYXFWI
j =  16
Task ID: 77UPLQ7GOUXXAUPYD4ZRZNHS
j =  17
i =  8
Task ID: I6JPOPZF4YIALTO4ZKKFTXV7
j =  12
Task ID: BSCBYZCXLIK5LE3ECKHHD5UB
j =  13
Task ID: S2OXLO76Y6A4T7BALS25JEJB
j =  14
Task ID: 5KZ2FLHTH72AGBI2C4YPAINI
j =  15
Task ID: MP2JHBB4J6KK3MY64GGIUC2K
j =  16
Task ID: SNJ4

In [32]:
print("Export finished!")
pyperclip.copy("\'satellital\': " + str(satellital_ids)+"," +"\n"
               "\'elevation\': " + str(elevation_ids) + "\n")


Export finished!


Waiting...
